<a href="https://colab.research.google.com/github/HarshESC/TalkData-Mobile-Demographics/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive 
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/"
%cd "/content/gdrive/My Drive/Final Capstone"

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import (
    Activation,
    Dropout,
    Flatten,
    Dense,
    BatchNormalization,
)
from tensorflow.keras import backend as K


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Final Capstone


In [2]:
master_df = pd.read_csv("master_clean.csv")
X = master_df.drop(columns=["num_group"])
y = master_df["num_group"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=13, stratify=y
)


In [3]:
X_test


,event_id,app_id,is_active,device_id,label_id_x,download_id,251,252,253,254,256,262,263,302,303,306,326,405,406,407,548,549,551,558,562,564,565,566,english_phone_brand_asus,english_phone_brand_coolpad,english_phone_brand_hammer,english_phone_brand_huawei,english_phone_brand_lshi,english_phone_brand_meizu,english_phone_brand_oppo,english_phone_brand_qiku,english_phone_brand_samsung,english_phone_brand_vivo,english_phone_brand_xiaomi,device_model_Galaxy Note 3,device_model_Galaxy Note 4,device_model_Galaxy S5,device_model_Galaxy S6,device_model_MI 3,device_model_MI 4,device_model_MX4,device_model_MX4 Pro,device_model_MX5,device_model_Mate 7,device_model_P8 Lite,device_model_P8max,device_model_R7 Plus,device_model_U3,device_model_X5SL,device_model_ZenFone 2,device_model_note顶配版,device_model_坚果手机,device_model_大神Note3,device_model_小米4C,device_model_小米note,device_model_红米2,device_model_红米2A,device_model_红米Note2,device_model_红米Note3,device_model_红米note,device_model_荣耀3C,device_model_荣耀6,device_model_荣耀6 Plus,device_model_荣耀7,device_model_荣耀7i,device_model_荣耀X2,device_model_荣耀畅玩4C,device_model_荣耀畅玩5X,device_model_超级手机1,device_model_超级手机1 Pro,device_model_超级手机1s,device_model_青春版,device_model_魅蓝NOTE,device_model_魅蓝metal
8010,241035,-329259848532828564,0,9200003785071800248,302,8870743936538971684,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8000,241035,-5305696816021977482,0,9200003785071800248,548,3894306969049822766,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5534,119250,7324914609217160422,0,-1540573801328129375,302,5784340807889031047,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1252,9494,-2601201025890389322,0,-2922663263004220160,405,-5523864288894609482,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4503,73440,7316250158002095415,0,2407726915124545999,251,-8722767000582910202,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3177,41529,-5720078949152207372,0,-1373869470418546394,302,-7093948419570753766,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6750,166584,-6098437695588462435,0,5907292383047496113,302,-191145312540966322,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5510,119250,-5720078949152207372,0,-1540573801328129375,302,-7260652750480336747,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8973,293941,7324843852388795447,0,-2845809539101591108,405,4479034313287204339,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
y_test


8010    1
8000    1
5534    0
1252    0
4503    1
       ..
3177    8
6750    0
5510    0
8973    4
3784    1
Name: num_group, Length: 2447, dtype: int64

In [5]:
master_df["num_group"].value_counts()


5     2842
4     2690
3     1366
1     1250
2      885
11     857
10     683
0      637
9      403
8      282
6      194
7      142
Name: num_group, dtype: int64

In [6]:
# Output_dim = 12 because there are 12 different age/gender groups
output_dim = 12


In [7]:
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
y_train = to_categorical(y_train, output_dim)
y_test = to_categorical(y_test, output_dim)


# 3 Layer RELU Model with ADAM Optimizer


In [8]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(12, activation="softmax"))


In [9]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [10]:
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1)


Epoch 1/20
77/77 [==============================] - 1s 2ms/step - loss: 144428256831995904.0000 - accuracy: 0.1612
Epoch 2/20
77/77 [==============================] - 0s 2ms/step - loss: 25122343898578944.0000 - accuracy: 0.1709
Epoch 3/20
77/77 [==============================] - 0s 2ms/step - loss: 20149549158891520.0000 - accuracy: 0.1740
Epoch 4/20
77/77 [==============================] - 0s 2ms/step - loss: 16967618342682624.0000 - accuracy: 0.1692
Epoch 5/20
77/77 [==============================] - 0s 2ms/step - loss: 14389785414074368.0000 - accuracy: 0.1702
Epoch 6/20
77/77 [==============================] - 0s 2ms/step - loss: 12388376825036800.0000 - accuracy: 0.1705
Epoch 7/20
77/77 [==============================] - 0s 2ms/step - loss: 12513431240310784.0000 - accuracy: 0.1792
Epoch 8/20
77/77 [==============================] - 0s 2ms/step - loss: 10716036819058688.0000 - accuracy: 0.1664
Epoch 9/20
77/77 [==============================] - 0s 2ms/step - loss: 100492227452600

In [11]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])


Test accuracy: 0.2149570882320404


In [12]:
y_pred = model.predict(X_test)
y_test_arg = tf.argmax(y_test, 1)
y_pred_arg = tf.argmax(y_pred, 1)


In [13]:
res = tf.math.confusion_matrix(y_test_arg, y_pred_arg)
print(res)


tf.Tensor(
[[  0   0   0  11   0  90   0   0   0   5  18   3]
 [  0   0   0  10   2 207   0   0   0   4  17  10]
 [  0   0   0  14   0 142   0   0   0   5  12   4]
 [  0   0   1  18   0 229   3   0   0   7  11   4]
 [  0   0   1  51  12 426   0   0   0  13  19  16]
 [  0   1   0  35   7 475   4   0   0   8  24  15]
 [  0   0   0   3   0  34   0   0   0   0   1   1]
 [  0   0   0   0   0  28   0   0   0   0   0   0]
 [  0   0   0   0   1  50   0   0   0   1   2   2]
 [  0   1   0   8   0  65   0   0   0   2   4   1]
 [  0   0   0  12   0 103   0   0   0   4  14   4]
 [  0   1   0  12   2 148   0   0   0   1   3   5]], shape=(12, 12), dtype=int32)


# 3 Layer RELU Model with SGD Optimizer


In [14]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(12, activation="softmax"))


In [15]:
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [16]:
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1)


Epoch 1/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0529
Epoch 2/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 3/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 4/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 5/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 6/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 7/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 8/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 9/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 10/20
77/77 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0521
Epoch 11/20
77/77 [==============================

In [17]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])


Test accuracy: 0.051900286227464676


In [18]:
y_pred = model.predict(X_test)
y_test_arg = tf.argmax(y_test, 1)
y_pred_arg = tf.argmax(y_pred, 1)


In [19]:
res = tf.math.confusion_matrix(y_test_arg, y_pred_arg)
print(res)


tf.Tensor(
[[127   0   0   0   0   0   0   0   0   0   0   0]
 [250   0   0   0   0   0   0   0   0   0   0   0]
 [177   0   0   0   0   0   0   0   0   0   0   0]
 [273   0   0   0   0   0   0   0   0   0   0   0]
 [538   0   0   0   0   0   0   0   0   0   0   0]
 [569   0   0   0   0   0   0   0   0   0   0   0]
 [ 39   0   0   0   0   0   0   0   0   0   0   0]
 [ 28   0   0   0   0   0   0   0   0   0   0   0]
 [ 56   0   0   0   0   0   0   0   0   0   0   0]
 [ 81   0   0   0   0   0   0   0   0   0   0   0]
 [137   0   0   0   0   0   0   0   0   0   0   0]
 [172   0   0   0   0   0   0   0   0   0   0   0]], shape=(12, 12), dtype=int32)


# 3 Layer TANH Model with ADAM Optimizer

In [20]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation="tanh"))
model.add(Dense(64, activation="tanh"))
model.add(Dense(12, activation="softmax"))


In [21]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [22]:
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1)


Epoch 1/20
77/77 [==============================] - 1s 2ms/step - loss: 2.4352 - accuracy: 0.1678
Epoch 2/20
77/77 [==============================] - 0s 2ms/step - loss: 2.2797 - accuracy: 0.2342
Epoch 3/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1743 - accuracy: 0.2529
Epoch 4/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1176 - accuracy: 0.2602
Epoch 5/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0921 - accuracy: 0.2673
Epoch 6/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0830 - accuracy: 0.2661
Epoch 7/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0687 - accuracy: 0.2713
Epoch 8/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0655 - accuracy: 0.2744
Epoch 9/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0566 - accuracy: 0.2722
Epoch 10/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0509 - accuracy: 0.2775
Epoch 11/20
77/77 [

In [23]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])


Test accuracy: 0.2738046646118164


In [24]:
y_pred = model.predict(X_test)
y_test_arg = tf.argmax(y_test, 1)
y_pred_arg = tf.argmax(y_pred, 1)


In [25]:
res = tf.math.confusion_matrix(y_test_arg, y_pred_arg)
print(res)


tf.Tensor(
[[  2  12   2  11  75  22   0   0   0   0   3   0]
 [  0  42   1  36 106  65   0   0   0   0   0   0]
 [  0  16   0  20  88  53   0   0   0   0   0   0]
 [  0  22   1  86 114  47   0   0   0   3   0   0]
 [  3  18   1  41 299 175   0   0   0   0   1   0]
 [  0  18   1  71 238 239   0   0   0   0   2   0]
 [  0   6   0  18  10   5   0   0   0   0   0   0]
 [  0   2   0  15   5   3   0   0   0   3   0   0]
 [  0   4   0   6  23  23   0   0   0   0   0   0]
 [  1   8   1   9  35  27   0   0   0   0   0   0]
 [  3  11   0  29  63  29   0   0   0   0   2   0]
 [  0   7   3  24  85  53   0   0   0   0   0   0]], shape=(12, 12), dtype=int32)


# 3 Layer TANH Model with SGD Optimizer

In [26]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation="tanh"))
model.add(Dense(64, activation="tanh"))
model.add(Dense(12, activation="softmax"))


In [27]:
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [28]:
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1)


Epoch 1/20
77/77 [==============================] - 1s 2ms/step - loss: 2.5571 - accuracy: 0.1143
Epoch 2/20
77/77 [==============================] - 0s 2ms/step - loss: 2.4017 - accuracy: 0.1705
Epoch 3/20
77/77 [==============================] - 0s 2ms/step - loss: 2.3347 - accuracy: 0.2065
Epoch 4/20
77/77 [==============================] - 0s 2ms/step - loss: 2.2853 - accuracy: 0.2212
Epoch 5/20
77/77 [==============================] - 0s 2ms/step - loss: 2.2469 - accuracy: 0.2322
Epoch 6/20
77/77 [==============================] - 0s 2ms/step - loss: 2.2171 - accuracy: 0.2417
Epoch 7/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1934 - accuracy: 0.2443
Epoch 8/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1744 - accuracy: 0.2472
Epoch 9/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1600 - accuracy: 0.2532
Epoch 10/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1484 - accuracy: 0.2538
Epoch 11/20
77/77 [

In [29]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])


Test accuracy: 0.2664487063884735


In [30]:
y_pred = model.predict(X_test)
y_test_arg = tf.argmax(y_test, 1)
y_pred_arg = tf.argmax(y_pred, 1)


In [31]:
res = tf.math.confusion_matrix(y_test_arg, y_pred_arg)
print(res)


tf.Tensor(
[[  0  12   0  13  65  37   0   0   0   0   0   0]
 [  0  31   0  30  80 108   0   0   0   0   1   0]
 [  0  14   0  13  67  83   0   0   0   0   0   0]
 [  0  23   0  78  74  98   0   0   0   0   0   0]
 [  0  21   0  34 251 231   0   0   0   0   1   0]
 [  0  24   0  74 179 292   0   0   0   0   0   0]
 [  0   7   0  18   5   9   0   0   0   0   0   0]
 [  0   2   0  15   6   5   0   0   0   0   0   0]
 [  0   1   0   6  12  37   0   0   0   0   0   0]
 [  0   4   0  10  30  37   0   0   0   0   0   0]
 [  0   8   0  27  50  52   0   0   0   0   0   0]
 [  0   6   0  22  59  85   0   0   0   0   0   0]], shape=(12, 12), dtype=int32)


# 3 Layer Sigmoid with ADAM Optimizer

In [36]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation="sigmoid"))
model.add(Dense(64, activation="sigmoid"))
model.add(Dense(12, activation="softmax"))


In [33]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [34]:
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1)


Epoch 1/20
77/77 [==============================] - 1s 2ms/step - loss: 2.2403 - accuracy: 0.2069
Epoch 2/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1431 - accuracy: 0.2392
Epoch 3/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1311 - accuracy: 0.2444
Epoch 4/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1218 - accuracy: 0.2522
Epoch 5/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1148 - accuracy: 0.2533
Epoch 6/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1090 - accuracy: 0.2558
Epoch 7/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1055 - accuracy: 0.2524
Epoch 8/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0998 - accuracy: 0.2567
Epoch 9/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0949 - accuracy: 0.2595
Epoch 10/20
77/77 [==============================] - 0s 2ms/step - loss: 2.0919 - accuracy: 0.2576
Epoch 11/20
77/77 [

In [35]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])


Test accuracy: 0.2664487063884735


In [37]:
y_pred = model.predict(X_test)
y_test_arg = tf.argmax(y_test, 1)
y_pred_arg = tf.argmax(y_pred, 1)


In [38]:
res = tf.math.confusion_matrix(y_test_arg, y_pred_arg)
print(res)


tf.Tensor(
[[  0   0   0 127   0   0   0   0   0   0   0   0]
 [  0   0   0 247   3   0   0   0   0   0   0   0]
 [  0   0   0 177   0   0   0   0   0   0   0   0]
 [  0   0   0 273   0   0   0   0   0   0   0   0]
 [  0   0   0 536   2   0   0   0   0   0   0   0]
 [  0   0   0 567   2   0   0   0   0   0   0   0]
 [  0   0   0  39   0   0   0   0   0   0   0   0]
 [  0   0   0  28   0   0   0   0   0   0   0   0]
 [  0   0   0  56   0   0   0   0   0   0   0   0]
 [  0   0   0  81   0   0   0   0   0   0   0   0]
 [  0   0   0 137   0   0   0   0   0   0   0   0]
 [  0   0   0 172   0   0   0   0   0   0   0   0]], shape=(12, 12), dtype=int32)


# 3 Layer Sigmoid with SGD Optimizer

In [39]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation="sigmoid"))
model.add(Dense(64, activation="sigmoid"))
model.add(Dense(12, activation="softmax"))


In [40]:
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [41]:
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1)


Epoch 1/20
77/77 [==============================] - 0s 2ms/step - loss: 2.3280 - accuracy: 0.1794
Epoch 2/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1784 - accuracy: 0.2241
Epoch 3/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1659 - accuracy: 0.2299
Epoch 4/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1618 - accuracy: 0.2326
Epoch 5/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1593 - accuracy: 0.2387
Epoch 6/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1576 - accuracy: 0.2371
Epoch 7/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1559 - accuracy: 0.2399
Epoch 8/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1549 - accuracy: 0.2380
Epoch 9/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1537 - accuracy: 0.2395
Epoch 10/20
77/77 [==============================] - 0s 2ms/step - loss: 2.1526 - accuracy: 0.2378
Epoch 11/20
77/77 [

In [42]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])


Test accuracy: 0.2341642826795578


In [43]:
y_pred = model.predict(X_test)
y_test_arg = tf.argmax(y_test, 1)
y_pred_arg = tf.argmax(y_pred, 1)


In [44]:
res = tf.math.confusion_matrix(y_test_arg, y_pred_arg)
print(res)


tf.Tensor(
[[  0   0   0   0  63  64   0   0   0   0   0   0]
 [  0   0   0   0  88 162   0   0   0   0   0   0]
 [  0   0   0   0  68 109   0   0   0   0   0   0]
 [  0   0   0   0  81 192   0   0   0   0   0   0]
 [  0   0   0   0 212 326   0   0   0   0   0   0]
 [  0   0   0   0 208 361   0   0   0   0   0   0]
 [  0   0   0   0  15  24   0   0   0   0   0   0]
 [  0   0   0   0   6  22   0   0   0   0   0   0]
 [  0   0   0   0  18  38   0   0   0   0   0   0]
 [  0   0   0   0  34  47   0   0   0   0   0   0]
 [  0   0   0   0  53  84   0   0   0   0   0   0]
 [  0   0   0   0  49 123   0   0   0   0   0   0]], shape=(12, 12), dtype=int32)


# 3 Layer RELU with ADAM Optimizer Smaller Batch Size and More Epochs

In [45]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(12, activation="softmax"))


In [46]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [47]:
model.fit(X_train, y_train, batch_size=64, epochs=100, verbose=1)


Epoch 1/100
153/153 [==============================] - 1s 2ms/step - loss: 143296292431331328.0000 - accuracy: 0.1557
Epoch 2/100
153/153 [==============================] - 0s 2ms/step - loss: 36241286230966272.0000 - accuracy: 0.1660
Epoch 3/100
153/153 [==============================] - 0s 2ms/step - loss: 32013354784522240.0000 - accuracy: 0.1642
Epoch 4/100
153/153 [==============================] - 0s 2ms/step - loss: 27367321156714496.0000 - accuracy: 0.1660
Epoch 5/100
153/153 [==============================] - 0s 2ms/step - loss: 26301417648029696.0000 - accuracy: 0.1754
Epoch 6/100
153/153 [==============================] - 0s 2ms/step - loss: 23619483302100992.0000 - accuracy: 0.1732
Epoch 7/100
153/153 [==============================] - 0s 2ms/step - loss: 21760267121590272.0000 - accuracy: 0.1717
Epoch 8/100
153/153 [==============================] - 0s 2ms/step - loss: 22288477232037888.0000 - accuracy: 0.1716
Epoch 9/100
153/153 [==============================] - 0s 2ms/s

In [48]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])


Test accuracy: 0.2325296252965927


In [49]:
y_pred = model.predict(X_test)
y_test_arg = tf.argmax(y_test, 1)
y_pred_arg = tf.argmax(y_pred, 1)


In [50]:
res = tf.math.confusion_matrix(y_test_arg, y_pred_arg)
print(res)


tf.Tensor(
[[  0   0   0   0   0 127   0   0   0   0   0   0]
 [  0   0   0   0   0 250   0   0   0   0   0   0]
 [  0   0   0   0   0 177   0   0   0   0   0   0]
 [  0   0   0   0   0 273   0   0   0   0   0   0]
 [  0   0   0   0   0 538   0   0   0   0   0   0]
 [  0   0   0   0   0 569   0   0   0   0   0   0]
 [  0   0   0   0   0  39   0   0   0   0   0   0]
 [  0   0   0   0   0  28   0   0   0   0   0   0]
 [  0   0   0   0   0  56   0   0   0   0   0   0]
 [  0   0   0   0   0  81   0   0   0   0   0   0]
 [  0   0   0   0   0 137   0   0   0   0   0   0]
 [  0   0   0   0   0 172   0   0   0   0   0   0]], shape=(12, 12), dtype=int32)
